In [16]:
import os
import config as config
from pathlib import Path
import hopsworks
from dotenv import load_dotenv

load_dotenv()
project = hopsworks.login(
    project=os.getenv("HOPSWORKS_PROJECT_NAME"),
    api_key_value=os.getenv("HOPSWORKS_API_KEY")
)
feature_store = project.get_feature_store()
feature_group=feature_store.get_or_create_feature_group(
    name=os.getenv("FEATURE_GROUP_NAME"),
    version=os.getenv("FEATURE_GROUP_VERSION"),
    description= "Time-series Data for Bike at six hour frequency",
    primary_key=["location_id","Pickup_hour"],
    event_time="Pickup_hour"
)

2025-05-10 18:04:24,053 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 18:04:24,184 INFO: Initializing external client
2025-05-10 18:04:24,186 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 18:04:24,892 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649


In [17]:
import hopsworks

api_key = os.getenv('HOPSWORKS_API_KEY')  
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')  

# pip install confluent-kafka
# Initialize connection to Hopsworks  
project = hopsworks.login(  
    api_key_value=api_key,  
    project=project_name  
)  
print(f"Successfully connected to Hopsworks project: {project_name}")

2025-05-10 18:04:25,509 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 18:04:25,529 INFO: Initializing external client
2025-05-10 18:04:25,532 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 18:04:26,090 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215649
Successfully connected to Hopsworks project: s3akash


In [18]:
feature_store = project.get_feature_store()

In [19]:
FEATURE_GROUP_NAME = "time_series_six_hourly_feature_group_bike"
FEATURE_GROUP_VERSION = 1

In [20]:
feature_group=feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description= "Time-series Data for Bike at six hour frequency",
    primary_key=["location_id","pickup_hour"],
    event_time="pickup_hour"
)

In [21]:
import pandas as pd
df = pd.read_parquet("transformeddata2024.parquet")
feature_group.insert(df,write_options={"wait_for_job":False})

Uploading Dataframe: 100.00% |██████████| Rows 2859/2859 | Elapsed Time: 00:04 | Remaining Time: 00:00


Launching job: time_series_six_hourly_feature_group_bike_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215649/jobs/named/time_series_six_hourly_feature_group_bike_1_offline_fg_materialization/executions


(Job('time_series_six_hourly_feature_group_bike_1_offline_fg_materialization', 'SPARK'),
 None)

In [22]:
df
df.info

<bound method DataFrame.info of              Pickup_hour location_id  target  target_lag_1  target_lag_2  \
0    2024-01-31 00:00:00     5329.03       2          50.0          76.0   
1    2024-01-31 06:00:00     5329.03      54           2.0          50.0   
2    2024-01-31 12:00:00     5329.03      72          54.0           2.0   
3    2024-01-31 18:00:00     5329.03      66          72.0          54.0   
4    2024-02-01 00:00:00     5329.03       4          66.0          72.0   
...                  ...         ...     ...           ...           ...   
2854 2024-09-29 18:00:00     6140.05       9          12.0           5.0   
2855 2024-09-30 00:00:00     6140.05       1           9.0          12.0   
2856 2024-09-30 06:00:00     6140.05      22           1.0           9.0   
2857 2024-09-30 12:00:00     6140.05      14          22.0           1.0   
2858 2024-09-30 18:00:00     6140.05      20          14.0          22.0   

      target_lag_3  target_lag_4  target_lag_5  target_

In [23]:
from dotenv import load_dotenv

load_dotenv()
# Create a feature view if it doesn't already exist
try:
    feature_store.create_feature_view(
        name=os.getenv('FEATURE_VIEW_NAME'),
        version=os.getenv('FEATURE_VIEW_VERSION'),
        query=feature_group.select_all(),
    )
    print(f"Feature view '{os.getenv('FEATURE_VIEW_NAME')}' (version {os.getenv('FEATURE_VIEW_VERSION')}) created successfully.")
except Exception as e:
    print(f"Error creating feature view: {e}")

# Retrieve the feature view
try:
    feature_view = feature_store.get_feature_view(
        name=os.getenv('FEATURE_VIEW_NAME'),
        version=os.getenv('FEATURE_VIEW_VERSION'),
    )
    print(f"Feature view '{os.getenv('FEATURE_VIEW_NAME')}' (version {os.getenv('FEATURE_VIEW_VERSION')}) retrieved successfully.")
except Exception as e:
    print(f"Error retrieving feature view: {e}")

Error creating feature view: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1215649/featurestores/1203280/featureview). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270179,"usrMsg":"Feature view: ime_series_six_hourly_feature_view_bike, version: 1","errorMsg":"The provided feature view name and version already exists"}', error code: 270179, error msg: The provided feature view name and version already exists, user msg: Feature view: ime_series_six_hourly_feature_view_bike, version: 1
Feature view 'ime_series_six_hourly_feature_view_bike' (version 1) retrieved successfully.
